In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-malaya/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

38052

In [5]:
files[0]

'output-malaya/2-0.json'

In [6]:
with open(files[0]) as fopen:
    d = json.load(fopen)

In [7]:
a = np.array(d[0]['predict_ms'])
a = a[a != 50257].tolist() + [50257]
t = tokenizer.decode(a, decode_with_timestamps = True).strip()
t

'<|startoftranscript|><|ms|><|transcribe|><|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan<|5.70|><|endoftext|>'

In [8]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [9]:
import itertools


def _pad_sequence(
    sequence,
    n,
    pad_left=False,
    pad_right=False,
    left_pad_symbol=None,
    right_pad_symbol=None,
):
    sequence = iter(sequence)
    if pad_left:
        sequence = itertools.chain((left_pad_symbol,) * (n - 1), sequence)
    if pad_right:
        sequence = itertools.chain(sequence, (right_pad_symbol,) * (n - 1))
    return sequence


def ngrams(
    sequence,
    n: int,
    pad_left=False,
    pad_right=False,
    left_pad_symbol=None,
    right_pad_symbol=None,
):
    """
    generate ngrams.

    Parameters
    ----------
    sequence : List[str]
        list of tokenize words.
    n : int
        ngram size

    Returns
    -------
    result: List[Tuple[str, str]]
    """
    sequence = _pad_sequence(
        sequence, n, pad_left, pad_right, left_pad_symbol, right_pad_symbol
    )

    history = []
    while n > 1:
        try:
            next_item = next(sequence)
        except StopIteration:
            return
        history.append(next_item)
        n -= 1
    for item in sequence:
        history.append(item)
        yield tuple(history)
        del history[0]

def remove_duplicate(string, n = 3):
    n = list(ngrams(string.split(), 3))
    already = set()
    dedup = []
    for n_ in n:
        n_ = ' '.join(n_)
        if n_ not in already:
            dedup.append(n_)
            already.add(n_)
    return ' '.join(dedup)

In [10]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [11]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [12]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = os.path.join('output-audio-malaya', f'{f_split}-{i}.mp3')
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
            
            if data[i]['score_ms'] > minimum_score:
                a = np.array(data[i]['predict_ms'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        try:
                            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                            repeat = (dense > 3).sum() > 1
                            if repeat:
                                t_ = remove_duplicate(t_)
                        except:
                            if len(t_) > 100:
                                t_ = remove_duplicate(t_)
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ms|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
                    
            
            if data[i]['score_en'] > minimum_score:
                a = np.array(data[i]['predict_en'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        try:
                            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                            repeat = (dense > 3).sum() > 1
                            if repeat:
                                t_ = remove_duplicate(t_)
                        except:
                            if len(t_) > 100:
                                t_ = remove_duplicate(t_)
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|en|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [13]:
# results = loop((files[:10], 0))

In [ ]:
results = mp.multiprocessing(files, loop, cores = 100)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/380 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/380 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/380 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/380 [00:00<?, ?it/s]Special tokens hav

In [15]:
len(results)

1089630

In [17]:
results[-4]

{'new_text': "<|startoftranscript|><|en|><|transcribe|><|0.00|> Allah Ta'ala<|1.44|><|1.44|> Jadikan kita<|2.34|><|2.34|> Kita bahasa Melayu<|5.20|><|5.20|> Sama ada bahasa Cina<|6.10|><|endoftext|>",
 'audio_filename': 'output-audio-malaya/0-9735-38.mp3'}

In [18]:
with open('prepared-pseudolabel-malaya.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 1089630/1089630 [00:03<00:00, 280322.10it/s]


In [19]:
from huggingface_hub import HfApi
api = HfApi()

In [20]:
api.upload_file(
    path_or_fileobj='prepared-pseudolabel-malaya.jsonl',
    path_in_repo='prepared-pseudolabel-malaya.jsonl',
    repo_id='mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3-timestamp',
    repo_type='dataset',
)

prepared-pseudolabel-malaya.jsonl:   0%|          | 0.00/309M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3-timestamp/blob/main/prepared-pseudolabel-malaya.jsonl'

In [21]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
tokenizer.tokenize(results[-4]['new_text'])

['<|startoftranscript|>',
 '<|en|>',
 '<|transcribe|>',
 '<|0.00|>',
 'ĠAllah',
 'ĠTa',
 "'",
 'ala',
 '<|1.44|>',
 '<|1.44|>',
 'ĠJ',
 'ad',
 'ikan',
 'Ġkita',
 '<|2.34|>',
 '<|2.34|>',
 'ĠKita',
 'Ġbah',
 'asa',
 'ĠMel',
 'ay',
 'u',
 '<|5.20|>',
 '<|5.20|>',
 'ĠS',
 'ama',
 'Ġada',
 'Ġbah',
 'asa',
 'ĠC',
 'ina',
 '<|6.10|>',
 '<|endoftext|>']